In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/Folders/Work/Research/CornellMovie/data/"


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [52]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchtext 
import numpy as np
import pandas as pd
from torchtext import data
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [53]:
X_train = []
Y_train = []
X_test = []
Y_test = []

In [54]:
with open(root_dir + 'training_set.txt', encoding="charmap") as training:
  for line in training:
    line = line.strip()
    line_num, chr_id, movie_id, chr_name, chr_gender, line_text, credit = line.split("+++$+++")
    if(chr_gender.strip().lower() == "m"):
      Y_train.append(0)
      X_train.append(line_text.strip())
    elif(chr_gender.strip().lower() == "f"):
      Y_train.append(1)
      X_train.append(line_text.strip())

with open(root_dir + 'test_set.txt', encoding="charmap") as test:
  for line in test:
    line = line.strip()
    line_num, chr_id, movie_id, chr_name, chr_gender, line_text, credit = line.split("+++$+++")
    if(chr_gender.strip().lower() == "m"):
      Y_test.append(0)
      X_test.append(line_text.strip())

    elif(chr_gender.strip().lower() == "f"):
      Y_test.append(1)
      X_test.append(line_text.strip())


In [55]:
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [56]:
X_test_counts = count_vect.fit_transform(X_test)
X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)

In [57]:
device = torch.device("cuda")

In [58]:
df = pd.DataFrame()

In [59]:
indices = []
for i in range(len(X_train)):
  if(len(X_train[i]) > 0):
    answer = True 
  else:
    indices.append(i)

for index in sorted(indices, reverse=True):
    del X_train[index]
    del Y_train[index]


In [60]:
y_indexes = []
for i in range(len(X_test)):
  if(len(X_test[i]) > 0):
    answer = True
  else:
    y_indexes.append(i)

for index in sorted(y_indexes, reverse=True):
  del X_test[index]
  del Y_test[index]

In [61]:
X = X_train + X_test
Y = Y_train + Y_test


from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(sampling_strategy='minority')
X, Y = oversample.fit_resample(np.array(X).reshape(-1,1), Y)

a = []
for phrase in X:
    a.append(phrase[0].strip())
X = a
df['text'] = X
df['target'] = Y 

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [62]:
def get_top_data(top_n = 30000):
  top_data_df_male = df[df['target'] == 0].head(top_n)
  top_data_df_female = df[df['target'] == 1].head(top_n)
  data_df_small = pd.concat([top_data_df_male, top_data_df_female])
  return data_df_small

In [63]:
top_data_df_small = df

In [64]:
from sklearn.model_selection import train_test_split
def split_train_test(top_data_df_small, test_size=0.1, shuffle_state=True):
    X_train, X_test, Y_train, Y_test = train_test_split(top_data_df_small[['text']], 
                                                        top_data_df_small['target'], 
                                                        shuffle=shuffle_state,
                                                        test_size=test_size, 
                                                        random_state=15)
    print("Value counts for Train genders")
    print(Y_train.value_counts())
    print("Value counts for Test genders")
    print(Y_test.value_counts())
    print(type(X_train))
    print(type(Y_train))
    X_train = X_train.reset_index()
    X_test = X_test.reset_index()
    Y_train = Y_train.to_frame()
    Y_train = Y_train.reset_index()
    Y_test = Y_test.to_frame()
    Y_test = Y_test.reset_index()
    print(X_train.head())
    return X_train, X_test, Y_train, Y_test

# Call the train_test_split
X_train, X_test, Y_train, Y_test = split_train_test(top_data_df_small)

Value counts for Train genders
0    139360
1    139238
Name: target, dtype: int64
Value counts for Test genders
1    15539
0    15417
Name: target, dtype: int64
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
    index                                               text
0   52967  Still scrubbing... looks like there's a third ...
1  307438                               Now just a minute...
2  243209                                       All I did...
3  115665                                       I know that.
4  138662  I'm okay.  Look, it's a big house. We needn't ...


In [65]:
test_df = pd.DataFrame()

In [66]:
len(X_test)

30956

In [67]:
len(Y_test)

30956

In [69]:
test_df['text'] = X_test['text']
test_df['target'] = Y_test['target']

In [70]:
test_df.head(10)

,text,target
0,"You've never been to a dance, have you?",0
1,In Dudley there were more sides of the tracks ...,0
2,"Well, I met someone...",1
3,It's you who's being hurt. By keeping it insid...,1
4,Don't be dense. The party.,1
5,Don't you open them anymore?,1
6,You know what? It's Christmas Eve. And you loo...,1
7,I'd soon die.,1
8,Yeah? See ya.,1
9,You know what? I think you guys are all jealous.,0


In [71]:
train_df = pd.DataFrame()
train_df['text'] = X_train['text']
train_df['target'] = Y_train['target']

In [72]:
train_df.head()

,text,target
0,Still scrubbing... looks like there's a third ...,0
1,Now just a minute...,1
2,All I did...,1
3,I know that.,0
4,"I'm okay. Look, it's a big house. We needn't ...",0


In [73]:
SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [74]:
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [75]:
class DataFrameDataset(data.Dataset):

    def __init__(self, df, fields, is_test=False, **kwargs):
        examples = []
        for i, row in df.iterrows():
            label = row.target if not is_test else None
            text = row.text
            examples.append(data.Example.fromlist([text, label], fields))

        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex):
        return len(ex.text)

    @classmethod
    def splits(cls, fields, train_df, val_df=None, test_df=None, **kwargs):
        train_data, val_data, test_data = (None, None, None)
        data_field = fields

        if train_df is not None:
            train_data = cls(train_df.copy(), data_field, **kwargs)
        if val_df is not None:
            val_data = cls(val_df.copy(), data_field, **kwargs)
        if test_df is not None:
            test_data = cls(test_df.copy(), data_field, True, **kwargs)

        return tuple(d for d in (train_data, val_data, test_data) if d is not None)

In [76]:
fields = [('text',TEXT), ('label',LABEL)]
train_ds, test_ds = DataFrameDataset.splits(fields, train_df=train_df, val_df=test_df)


In [77]:
MAX_VOCAB_SIZE = len(X_train)

TEXT.build_vocab(train_ds, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = 'glove.6B.200d',
                 unk_init = torch.Tensor.zero_)

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                           
100%|█████████▉| 399578/400000 [00:36<00:00, 10110.04it/s]

In [78]:
LABEL.build_vocab(train_ds)

In [79]:
BATCH_SIZE = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_ds, test_ds), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [80]:
# Hyperparameters
num_epochs = 25
learning_rate = 0.001

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 200
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 4
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] # padding

In [81]:
class LSTM_net(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        
        self.fc2 = nn.Linear(hidden_dim, 1)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        # text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        # embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        # output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        # output = [sent len, batch size, hid dim * num directions]
        # output over padding tokens are zero tensors
        
        # hidden = [num layers * num directions, batch size, hid dim]
        # cell = [num layers * num directions, batch size, hid dim]
        
        # concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        # and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        output = self.fc1(hidden)
        output = self.dropout(self.fc2(output))
                
        #hidden = [batch size, hid dim * num directions]
            
        return output

In [82]:
#creating instance of our LSTM_net class

model = LSTM_net(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [83]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)
model.embedding.weight.data.copy_(pretrained_embeddings)

torch.Size([54837, 200])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1229,  0.5804, -0.0696,  ..., -0.0392, -0.1624, -0.0967],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [84]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1229,  0.5804, -0.0696,  ..., -0.0392, -0.1624, -0.0967],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])


In [85]:
model.to(device) #CNN to GPU


# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)


100%|█████████▉| 399578/400000 [00:50<00:00, 10110.04it/s]

In [86]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [87]:
def train(model, iterator):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        text, text_lengths = batch.text
        # for a in text_lengths:
        #   if(a.item <= 0):
        #     text_lengths.remove
        optimizer.zero_grad()
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)

        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [88]:
def evaluate(model, iterator):
    
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text

            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [89]:
import time

t = time.time()
loss=[]
acc=[]
val_acc=[]
val_losses=[]

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_iterator)
    val_loss, valid_acc = evaluate(model, valid_iterator)
    print("Epoch " + str(epoch) + " :")
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\tVal Loss: {val_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')
    print('\n')
    loss.append(train_loss)
    acc.append(train_acc)
    val_acc.append(valid_acc)
    val_losses.append(val_loss)
print(f'time:{time.time()-t:.3f}')

Epoch 0 :
	Train Loss: 0.682 | Train Acc: 53.76%
	Val Loss: 0.654 | Val. Acc: 61.93%


Epoch 1 :
	Train Loss: 0.653 | Train Acc: 57.75%
	Val Loss: 0.623 | Val. Acc: 64.64%


Epoch 2 :
	Train Loss: 0.631 | Train Acc: 59.12%
	Val Loss: 0.611 | Val. Acc: 66.00%


Epoch 3 :
	Train Loss: 0.613 | Train Acc: 60.35%
	Val Loss: 0.592 | Val. Acc: 67.03%


Epoch 4 :
	Train Loss: 0.597 | Train Acc: 61.25%
	Val Loss: 0.583 | Val. Acc: 67.90%


Epoch 5 :
	Train Loss: 0.584 | Train Acc: 62.07%
	Val Loss: 0.569 | Val. Acc: 69.15%


Epoch 6 :
	Train Loss: 0.571 | Train Acc: 62.75%
	Val Loss: 0.557 | Val. Acc: 69.82%


Epoch 7 :
	Train Loss: 0.560 | Train Acc: 63.45%
	Val Loss: 0.549 | Val. Acc: 70.09%


Epoch 8 :
	Train Loss: 0.549 | Train Acc: 64.21%
	Val Loss: 0.543 | Val. Acc: 71.05%


Epoch 9 :
	Train Loss: 0.540 | Train Acc: 64.65%
	Val Loss: 0.533 | Val. Acc: 71.84%


Epoch 10 :
	Train Loss: 0.530 | Train Acc: 65.06%
	Val Loss: 0.541 | Val. Acc: 71.63%


Epoch 11 :
	Train Loss: 0.522 | Train Acc: